In [1]:
import pandas as pd

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

sns.set(context='poster')
%matplotlib inline

### Задача 1:

Рассмотрим как можно провести отбор признаков с помощью обучения линейной регрессии и леса с малым количеством деревьев на примере задачи регрессии. Будем использовать датасет <https://www.kaggle.com/datasets/abrambeyer/openintro-possum> и пытаться предсказать возраст оппосумов на основе различных параметров их тела.

Для упрощения задачи избавимся от категориальных признаков

In [2]:
data = pd.read_csv('possum.csv').dropna()
X = data[data.columns.drop(['sex', 'Pop', 'age'])]
y = data['age']

Разделим выборку на тренировочную и тестовую часть

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Стандартизируем данные для корректной работы линейной регрессии:

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
scaler = StandardScaler()
scaler.fit(X_test)
X_test_norm = scaler.transform(X_test)

Обучим модель линейной регрессии с L1-регуляризацией на всех данных, посмотрим на метрику качества

In [23]:
lin_model = Lasso(alpha=0.1)
lin_model.fit(X_train_norm,y_train)
print('Коэффиценты модели:', lin_model.coef_)
print('Значение MSE на тренировочной выборке:', MSE(lin_model.predict(X_train_norm), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model.predict(X_test_norm), y_test))

Коэффиценты модели: [-0.          0.          0.2451365   0.0463782   0.         -0.
 -0.07870314  0.          0.23833098  0.05802872  0.30598817]
Значение MSE на тренировочной выборке: 3.090739258572398
Значение MSE на тестовой выборке: 2.8270484069672555


Проделаем аналогичную процедуру с лесом c небольшим количеством деревьев:

In [24]:
forest = RandomForestRegressor()
forest.fit(X_train_norm,y_train)
print('Важность признаков:', forest.feature_importances_)
print('Значение MSE на тренировочной выборке:', MSE(forest.predict(X_train), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest.predict(X_test), y_test))

Важность признаков: [0.11011793 0.0084274  0.23463091 0.21084882 0.06611679 0.06476611
 0.06272389 0.06829238 0.06871246 0.03602021 0.06934312]
Значение MSE на тренировочной выборке: 3.7091985074626868
Значение MSE на тестовой выборке: 3.4789000000000003


C:\Users\AsusPro\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
C:\Users\AsusPro\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


Составим 2 списка наиболее важных признаков - один с признаками, наиболее важными для линейной регрессии, второй с наиболее важными для леса

In [25]:
lin_imp = []
forest_imp = []
for i, column in enumerate(X.columns):
    if lin_model.coef_[i] > 0.1:
        lin_imp.append(i)
    if forest.feature_importances_[i] > 0.1:
        forest_imp.append(column)

Теперь обучим модели только на важных признаках:

In [26]:
X_train_norm_lin_imp = X_train_norm[:, lin_imp]
X_test_norm_lin_imp = X_test_norm[:, lin_imp]

X_train_forest_imp = X_train[forest_imp]
X_test_forest_imp = X_test[forest_imp]

In [27]:
lin_model_2 = Lasso(alpha=0.1).fit(X_train_norm_lin_imp, y_train)
print('Коэффиценты модели:', lin_model_2.coef_)
print('Значение MSE на тренировочной выборке:', MSE(lin_model_2.predict(X_train_norm_lin_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model_2.predict(X_test_norm_lin_imp), y_test))

Коэффиценты модели: [0.27942877 0.23624623 0.30994129]
Значение MSE на тренировочной выборке: 3.1274779658965923
Значение MSE на тестовой выборке: 2.8264394617668707


In [28]:
forest_2 = RandomForestRegressor().fit(X_train_forest_imp, y_train)
print('Важность признаков:', forest_2.feature_importances_)
print('Значение MSE на тренировочной выборке:', MSE(forest_2.predict(X_train_forest_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest_2.predict(X_test_forest_imp), y_test))

Важность признаков: [0.25741675 0.34372405 0.3988592 ]
Значение MSE на тренировочной выборке: 0.4045074626865671
Значение MSE на тестовой выборке: 2.395323529411765


**Вывод:** Отсеивание признаков ведет к росту точности модели.

Для различных моделей важные признаки - разные